<a href="https://colab.research.google.com/github/erodriguezda/senales2025/blob/main/taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Instalar librerías y herramientas necesarias
!apt-get update -qq && apt-get install -y ffmpeg
!pip install -q streamlit numpy scipy matplotlib yt-dlp pydub sympy

# 2. Descargar Cloudflared para el túnel de acceso
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 67 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.6 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, ifft, fftfreq, fftshift
from scipy.io import wavfile
import sympy as sp
import yt_dlp
import os
import subprocess


st.set_page_config(page_title="Taller 2 - Laplace", page_icon="📡", layout="wide")


plt.style.use('dark_background')


st.markdown("""
    <style>
    /* Fuente global */
    html, body, [class*="css"] {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }

    /* Fondo Principal */
    .stApp {
        background-color: #1e1e2f;
        color: #ffffff;
    }

    /* Encabezados */
    h1 {
        color: #00d4ff !important; /* Cyan neón */
        text-shadow: 0px 0px 10px rgba(0, 212, 255, 0.5);
    }
    h2 {
        color: #ffd700 !important; /* Dorado */
        border-bottom: 2px solid #ffd700;
        padding-bottom: 10px;
    }
    h3 {
        color: #ff6b6b !important; /* Rojo suave */
    }

    /* Sidebar */
    [data-testid="stSidebar"] {
        background-color: #27293d;
        border-right: 1px solid #444;
    }

    /* Botones */
    .stButton>button {
        background-color: #00d4ff;
        color: #000000;
        border-radius: 20px;
        font-weight: bold;
        border: none;
        transition: transform 0.2s;
    }
    .stButton>button:hover {
        background-color: #ffffff;
        transform: scale(1.05);
    }

    /* Alertas e Info */
    .stAlert {
        background-color: #27293d;
        color: #fff;
        border: 1px solid #555;
    }
    </style>
""", unsafe_allow_html=True)

# --- FUNCIONES AUXILIARES ---
@st.cache_data
def descargar_audio(url, start, dur=5):
    """Descarga audio de YT, recorta y normaliza."""
    filename = "temp_audio.wav"
    opts = {'format': 'bestaudio/best', 'outtmpl': 'temp_raw', 'quiet': True, 'overwrites': True}
    try:
        if os.path.exists(filename): os.remove(filename)
        with yt_dlp.YoutubeDL(opts) as ydl:
            ydl.download([url])
        subprocess.run(['ffmpeg', '-i', 'temp_raw', '-ss', str(start), '-t', str(dur), '-ac', '1', '-ar', '44100', filename, '-y'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        if not os.path.exists(filename): return None, None
        fs, data = wavfile.read(filename)
        if data.dtype != np.float32: data = data.astype(np.float32) / np.max(np.abs(data))
        return fs, data
    except: return None, None

def plot_freq_resp(w, mag, phase, title):
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 5))

    ax1.semilogx(w, mag, color='#00d4ff', linewidth=2)
    ax1.set_title(title, color='white')
    ax1.set_ylabel("Magnitud (dB)", color='white')
    ax1.grid(True, which="both", alpha=0.3)

    ax2.semilogx(w, phase, color='#ff6b6b', linewidth=2)
    ax2.set_ylabel("Fase (rad)", color='white')
    ax2.set_xlabel("Frecuencia (rad/s)", color='white')
    ax2.grid(True, which="both", alpha=0.3)
    return fig



def vista_inicio():

    st.markdown("""
    <div style='background-color:#27293d;padding:30px;border-radius:15px;text-align:center;border:1px solid #444;box-shadow: 0 4px 8px rgba(0,0,0,0.3);'>
        <h1 style='color:#00d4ff;margin-bottom:0;'>Taller 2: Señales, Sistemas y Laplace</h1>
        <h3 style='color:#ffd700;margin-top:10px;'>Solución Interactiva</h3>
        <p style='color:#bbb;'>Universidad Nacional de Colombia</p>
    </div>
    """, unsafe_allow_html=True)
    st.markdown("""
    ### 📂 Contenido del Taller:
    * **📡 1. Modulación AM (1.6)**: Transmisión y demodulación coherente.
    * **⚡ 2. Potencia y THD (1.7)**: Análisis de rectificador y armónicos.
    * **⚙️ 3. Sistemas SLIT (2.3)**: Simulación de linealidad e invarianza.
    * **📈 4. Convolución (2.4)**: Respuesta a impulsos y escalones discretos.
    * **🔀 5. Sistemas en Serie (2.5)**: Análisis de no linealidad en cascada.
    * **📐 6. Laplace y ROC (2.9)**: Polos, ceros y regiones de convergencia.
    * **🔌 7. Circuitos RLC (2.12-2.14)**: Análisis completo (Bode, Step, Control).
    """)

def vista_am():
    st.header("📡 1. Modulación y Demodulación AM (Punto 1.6)")
    st.info("Implementación del diagrama: Mezclador $\\rightarrow$ LPF (Ideal) $\\rightarrow$ Escalamiento.")

    col1, col2 = st.columns([1, 2])
    with col1:
        url = st.text_input("URL YouTube", "https://www.youtube.com/watch?v=dQw4w9WgXcQ")
        start = st.number_input("Inicio (s)", 20)
        fc = st.slider("Portadora (Hz)", 1000, 15000, 5000)

    if st.button("▶️ Procesar"):
        with st.spinner("Descargando..."):
            fs, m_t = descargar_audio(url, start)

        if m_t is not None:
            t = np.linspace(0, len(m_t)/fs, len(m_t))
            # Modulación
            c_t = np.cos(2*np.pi*fc*t)
            y_t = (1 + m_t) * c_t # Asumiendo m=1 y Ac=1 normalizados

            # Demodulación
            # 1. Mezclador
            v_mix = y_t * np.cos(2*np.pi*fc*t) # Coherente theta=0

            # 2. Filtro Ideal (FFT)
            V_F = fft(v_mix)
            freqs = fftfreq(len(t), 1/fs)
            # Corte en fc/2 o 4kHz (audio)
            mask = np.abs(freqs) < 4000
            v_lpf = np.real(ifft(V_F * mask))

            # 3. Escalar (x2 para recuperar amplitud tras mixer, -DC component if needed)
            v_out = 2 * v_lpf
            v_out = v_out - np.mean(v_out) # Quitar DC del "1" de la portadora

            st.subheader("Resultados")
            c1, c2, c3 = st.columns(3)
            c1.markdown("**Mensaje Original**"); c1.audio(m_t, sample_rate=fs)
            c2.markdown("**Modulada AM**"); c2.audio(y_t, sample_rate=fs)
            c3.markdown("**Demodulada**"); c3.audio(v_out, sample_rate=fs)

            fig, ax = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
            ax[0].plot(t[:1000], m_t[:1000], color='#00d4ff'); ax[0].set_title("Mensaje (Zoom)")
            ax[1].plot(t[:1000], y_t[:1000], color='#ffd700'); ax[1].set_title("Modulada (Zoom)")
            ax[2].plot(t[:1000], v_out[:1000], color='#ff6b6b'); ax[2].set_title("Recuperada (Zoom)")
            st.pyplot(fig)

def vista_thd():
    st.header("⚡ 2. Circuitos: THD y Rectificador (Punto 1.7)")
    st.markdown("Simulación de Rectificador Onda Completa con carga RC.")

    R = st.slider("Resistencia R (Ω)", 10, 1000, 100)
    C_uF = st.slider("Capacitor C (µF)", 0, 1000, 100)
    C = C_uF * 1e-6
    f = 60

    t = np.linspace(0, 0.1, 10000)
    vin = 170 * np.sin(2*np.pi*f*t) # 120Vrms -> 170Vp
    vrect = np.abs(vin)

    # Simulación simple de descarga RC
    vout = np.zeros_like(vrect)
    vc = 0
    dt = t[1] - t[0]
    for i in range(len(t)):
        if vrect[i] > vc:
            vc = vrect[i]
        else:
            if C > 0: vc = vc * np.exp(-dt/(R*C))
            else: vc = vrect[i]
        vout[i] = vc

    # THD Cálculo (Sobre componente AC)
    # Fundamental del rizado es 2*f (120Hz)
    yf = fft(vout)
    xf = fftfreq(len(t), dt)
    # Tomar solo positivos
    mask_pos = xf > 0
    yf_mag = 2.0/len(t) * np.abs(yf[mask_pos])
    xf_pos = xf[mask_pos]

    # Fundamental aprox en 120Hz (onda completa)
    idx_fund = np.argmin(np.abs(xf_pos - 120))
    amp_fund = yf_mag[idx_fund]

    # Armónicos (240, 360, etc.)
    harmonics_pwr = 0
    for h in range(2, 10):
        idx = np.argmin(np.abs(xf_pos - h*120))
        harmonics_pwr += yf_mag[idx]**2

    thd = (np.sqrt(harmonics_pwr) / amp_fund) * 100 if amp_fund > 0 else 0

    c1, c2 = st.columns(2)
    c1.metric("THD (Voltaje Salida)", f"{thd:.2f} %")
    c2.metric("VDC Promedio", f"{np.mean(vout):.2f} V")

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t[:3000], vrect[:3000], '--', color='gray', alpha=0.5, label='Rectificado Puro')
    ax.plot(t[:3000], vout[:3000], '#ff6b6b', label='Salida RC')
    ax.set_title("Voltaje de Salida"); ax.legend(); ax.grid(True, alpha=0.3)
    st.pyplot(fig)

def vista_slit():
    st.header("⚙️ 3. Simulación de Sistemas (Punto 2.3)")
    system_opt = st.selectbox("Sistema a Analizar", [
        "1. y[n] = x[n]/3 + 2x[n-1] - y[n-1]",
        "2. y[n] = sum(x[k]^2)",
        "3. y[n] = median(x[n])",
        "4. y(t) = Ax(t) + B"
    ])

    st.write(f"**Sistema seleccionado:** {system_opt}")
    st.info("Para verificar Linealidad: $T(ax_1 + bx_2) = aT(x_1) + bT(x_2)$")

    # Test simple
    n = np.arange(0, 20)
    x1 = np.cos(0.2*np.pi*n)
    x2 = (n > 5).astype(float)
    a, b = 2, 3
    x_comb = a*x1 + b*x2

    if "1." in system_opt: # Recurrente
        def sys1(x):
            y = np.zeros_like(x)
            for i in range(1, len(x)): y[i] = x[i]/3 + 2*x[i-1] - y[i-1]
            return y
        func = sys1
        st.success("Resultado: LINEAL e INVARIANTE (SLIT)")

    elif "2." in system_opt: # Suma cuadrados
        def sys2(x): return np.cumsum(x**2)
        func = sys2
        st.error("Resultado: NO LINEAL (Cuadrático)")

    elif "3." in system_opt: # Mediana
        def sys3(x): return signal.medfilt(x, 3)
        func = sys3
        st.error("Resultado: NO LINEAL (Mediana)")

    elif "4." in system_opt: # Ax + B
        A, B = 2, 5
        def sys4(x): return A*x + B
        func = sys4
        st.warning("Resultado: NO LINEAL si B != 0 (Falla superposición)")

    # Gráficas de comprobación
    y_comb = func(x_comb)
    y_sum = a*func(x1) + b*func(x2)

    fig, ax = plt.subplots()
    ax.plot(y_comb, color='#00d4ff', lw=3, label='T(ax1 + bx2)')
    ax.plot(y_sum, color='#ff6b6b', ls='--', lw=2, label='aT(x1) + bT(x2)')
    ax.legend(); ax.set_title("Test de Linealidad"); ax.grid(True, alpha=0.3)
    st.pyplot(fig)

def vista_convolucion():
    st.header("📈 4. Convolución Discreta (Punto 2.4)")

    # Definición de señales
    x = np.array([-15, 5, -3, 0, 5, 7, -1])
    nx = np.arange(-2, 5)

    h = np.array([1, -2, 0, 1, -2])
    nh = np.arange(-2, 3)

    # Convolución y[n]
    y = np.convolve(x, h)
    ny = np.arange(nx[0]+nh[0], nx[-1]+nh[-1]+1)

    # Parte 2: Respuesta al escalón (opcional lógica mantenida)
    s_step = np.array([-1, 6, -10, 3, 1, -10, 2, 5])
    h_from_s = np.diff(s_step, prepend=0)
    y_step_case = np.convolve(x, h_from_s)

    st.subheader("Caso 1: Convolución Directa")
    c1, c2, c3 = st.columns(3)
    c1.write(f"x[n]: {x}"); c2.write(f"h[n]: {h}"); c3.write(f"y[n]: {y}")

    fig, ax = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    ax[0].stem(nx, x, linefmt='#00d4ff'); ax[0].set_title("Entrada x[n]"); ax[0].grid(True, alpha=0.3)
    ax[1].stem(nh, h, linefmt='#ff6b6b'); ax[1].set_title("Impulso h[n]"); ax[1].grid(True, alpha=0.3)
    ax[2].stem(ny, y, linefmt='#ffd700'); ax[2].set_title("Salida y[n]"); ax[2].grid(True, alpha=0.3)
    st.pyplot(fig)

def vista_serie():
    st.header("🔀 5. Sistemas en Serie (Punto 2.5)")
    st.markdown("Análisis de cascada con sistema No Lineal $y_A = x^2$ y LTI $h_B$.")

    st.latex(r"x(t) = e^{-at^2} \quad h_B(t) = B e^{-bt^2}")
    st.markdown("### Caso a) $x \to h_B \to y_A \to y$")
    st.latex(r"z(t) = x * h_B \quad (\text{Gaussiana} * \text{Gaussiana})")
    st.latex(r"y(t) = (z(t))^2")

    st.markdown("### Caso b) $x \to y_A \to h_B \to y$")
    st.latex(r"w(t) = x^2(t) = e^{-2at^2}")
    st.latex(r"y(t) = w(t) * h_B(t)")

    st.warning("⚠️ Debido a la no linealidad de A, el orden SÍ altera el resultado. (No conmuta).")

def vista_laplace():
    st.header("📐 6. Laplace y ROC (Punto 2.9)")

    opcion = st.selectbox("Señal", [
        "i) Causal: exp(-2t) + exp(-3t)",
        "ii) Bilateral/Mixta: exp(2t)u(t) + exp(-3t)u(-t)",
        "iii) Bilateral simétrica: exp(-a|t|)"
    ])

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.axhline(0, color='white'); ax.axvline(0, color='white')
    ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.grid(True, alpha=0.3)

    if "i)" in opcion:
        st.latex(r"X(s) = \frac{1}{s+2} + \frac{1}{s+3} = \frac{2s+5}{(s+2)(s+3)}")
        st.write("Polos: -2, -3. Causal -> ROC: Re(s) > -2")
        ax.axvline(-2, color='#ff6b6b', ls='--'); ax.axvline(-3, color='#ff6b6b', ls='--')
        ax.fill_betweenx([-5, 5], -2, 5, color='#00d4ff', alpha=0.3)
        ax.plot([-2, -3], [0, 0], 'rx', ms=10)

    elif "ii)" in opcion:
        st.latex(r"X(s) = \frac{1}{s-2} - \frac{1}{s+3}")
        st.write("Polos: 2, -3. ROC: -3 < Re(s) < 2")
        ax.axvline(2, color='#ff6b6b', ls='--'); ax.axvline(-3, color='#ff6b6b', ls='--')
        ax.fill_betweenx([-5, 5], -3, 2, color='#00d4ff', alpha=0.3)
        ax.plot([2, -3], [0, 0], 'rx', ms=10)

    elif "iii)" in opcion:
        st.latex(r"X(s) = \frac{-2a}{s^2 - a^2}")
        a = st.slider("Valor de a", 1.0, 4.0, 2.0)
        st.write(f"Polos: {a}, {-a}. ROC: {-a} < Re(s) < {a}")
        ax.axvline(a, color='#ff6b6b', ls='--'); ax.axvline(-a, color='#ff6b6b', ls='--')
        ax.fill_betweenx([-5, 5], -a, a, color='#00d4ff', alpha=0.3)
        ax.plot([a, -a], [0, 0], 'rx', ms=10)

    st.pyplot(fig)

def vista_rlc():
    st.header("🔌 7. Circuitos RLC (Puntos 2.12 - 2.14)")

    tipo = st.radio("Configuración", ["Serie (Salida Vc)", "Paralelo (Salida IL)"])
    damp = st.selectbox("Amortiguamiento", ["Subamortiguado", "Crítico", "Sobreamortiguado"])

    wn = st.slider("Frecuencia Natural (rad/s)", 1.0, 20.0, 5.0)

    if damp == "Subamortiguado": zeta = st.slider("Zeta", 0.1, 0.9, 0.5)
    elif damp == "Crítico": zeta = 1.0
    else: zeta = st.slider("Zeta", 1.1, 5.0, 2.0)

    C = 1e-6 # Fijo 1uF
    L = 1 / (wn**2 * C)

    if tipo == "Serie (Salida Vc)":
        R = 2 * zeta * wn * L
        st.latex(r"H(s) = \frac{\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2}")
    else:
        R = 1 / (2 * zeta * wn * C)
        st.latex(r"H(s) = \frac{\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2}")

    st.write(f"**Parámetros Calculados:** R={R:.2f}Ω, L={L*1000:.2f}mH, C=1µF")

    # Sistema
    num = [wn**2]
    den = [1, 2*zeta*wn, wn**2]
    sys = signal.TransferFunction(num, den)

    # Simulaciones
    t = np.linspace(0, 10, 500)
    t_imp, y_imp = signal.impulse(sys, T=t)
    t_step, y_step = signal.step(sys, T=t)

    # Gráficas
    tab1, tab2, tab3 = st.tabs(["Respuesta Tiempo", "Bode", "Polos y Ceros"])

    with tab1:
        fig, ax = plt.subplots(2, 1, figsize=(8, 6))
        ax[0].plot(t_step, y_step, '#00d4ff'); ax[0].set_title("Respuesta al Escalón"); ax[0].grid(True, alpha=0.3)
        ax[1].plot(t_imp, y_imp, '#ff6b6b'); ax[1].set_title("Respuesta al Impulso"); ax[1].grid(True, alpha=0.3)
        st.pyplot(fig)

    with tab2:
        w, mag, phase = signal.bode(sys)
        st.pyplot(plot_freq_resp(w, mag, phase, "Diagrama de Bode"))

    with tab3:
        fig, ax = plt.subplots()
        poles = sys.poles
        ax.plot(np.real(poles), np.imag(poles), 'rx', ms=10, label='Polos')
        ax.axhline(0, color='white'); ax.axvline(0, color='white'); ax.grid(True, alpha=0.3)
        ax.legend(); ax.set_title("Plano S")
        st.pyplot(fig)

# --- NAVEGACIÓN PRINCIPAL ---
def main():
    with st.sidebar:
        # st.image solo si la URL es válida, sino texto
        st.markdown("## 📡 Lab DSP")
        menu = st.radio("Ir a:", [
            "Inicio",
            "1. Modulación AM",
            "2. Potencia THD",
            "3. Sistemas SLIT",
            "4. Convolución Discreta",
            "5. Sistemas en Serie",
            "6. Laplace ROC",
            "7. Circuitos RLC"
        ])

    if menu == "Inicio": vista_inicio()
    elif menu == "1. Modulación AM": vista_am()
    elif menu == "2. Potencia THD": vista_thd()
    elif menu == "3. Sistemas SLIT": vista_slit()
    elif menu == "4. Convolución Discreta": vista_convolucion()
    elif menu == "5. Sistemas en Serie": vista_serie()
    elif menu == "6. Laplace ROC": vista_laplace()
    elif menu == "7. Circuitos RLC": vista_rlc()

if __name__ == '__main__':
    main()

Writing app.py


In [ ]:
# Ejecutar Streamlit en segundo plano
!streamlit run app.py &>/dev/null &

# Esperar un momento y lanzar el túnel
import time
time.sleep(3)
print("🚀 Dashboard disponible en:")
!cloudflared tunnel --url http://localhost:8501

🚀 Dashboard disponible en:
2025-12-09T03:26:34Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-09T03:26:34Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-09T03:26:37Z INF +--------------------------------------------------------------------------------------------+
2025-12-09T03:26:37Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-09T03:26:37Z INF |  https://moments-hottest-id